In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime

In [7]:
import pandas as pd
from io import StringIO
import requests
import datetime
class CrawlStockPriceTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime("%Y%m%d")
        self.target_name = "台股每日交易資訊"
        self.sub_market = ["sii", "otc", "rotc"]

    def crawl_sii(self):
        r = requests.post(
            "http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=" + self.date_str + "&type=ALLBUT0999")
        content = r.text.replace("=", "")
        lines = content.split("\n")
        lines = list(filter(lambda l: len(l.split('",')) > 10, lines))
        content = "\n".join(lines)
        if content == "":
            return None
        df = pd.read_csv(StringIO(content))
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(",", ""))
        df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df["date"] = pd.to_datetime(self.date).date()
        df = df[["證券代號", "date", "證券名稱", "成交股數", "成交金額", "開盤價", "收盤價", "最高價", "最低價"]]
        df = df.rename(columns={"證券代號": "stock_id", "證券名稱": "stock_name",
                                "成交股數": "turnover_vol", "成交金額": "turnover_price",
                                "開盤價": "open_price", "收盤價": "close_price",
                                "最高價": "high_price", "最低價": "low_price"})
        df['market'] = 'twse'
        return df

    @staticmethod
    def select_otc_id(code):
        if len(code) > 5:
            if code[-1] == "P":
                return False
            else:
                try:
                    code = int(code[:5])
                    if code > 10000:
                        return False
                    else:
                        return True
                except ValueError:
                    return True
        else:
            return True

    def crawl_otc(self):
        y = str(int(self.date.strftime("%Y")) - 1911)
        date_str = y + "/" + self.date.strftime("%m") + "/" + self.date.strftime("%d")
        link = "http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d=" \
               + date_str + "&s=0,asc,0"
        r = requests.get(link)
        lines = r.text.replace("\r", "").split("\n")
        try:
            df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
            df = df.astype(str)
        except pd.errors.ParserError:
            return None
        df.columns = list(map(lambda s: s.replace(" ", ""), lines[2].split(",")))
        df = df.apply(lambda s: s.str.replace(",", ""))
        df["stock_id"] = df["代號"]
        df["代號"] = df["代號"].apply(lambda s: self.select_otc_id(s))
        df = df[df["代號"]]
        df["date"] = pd.to_datetime(self.date).date()
        df = df[["stock_id", "date", "名稱", "成交股數", "成交金額(元)", "開盤", "收盤", "最高", "最低"]]
        df = df.rename(columns={"名稱": "stock_name",
                                "成交股數": "turnover_vol", "成交金額(元)": "turnover_price",
                                "開盤": "open_price", "收盤": "close_price",
                                "最高": "high_price", "最低": "low_price"})
        df.iloc[:, 3:] = df.iloc[:, 3:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df = df[df["turnover_vol"] >= 0]
        df['market'] = 'otc'
        return df

    def crawl_rotc(self):
        link = "http://www.tpex.org.tw/web/emergingstock/historical/daily/EMDaily_dl.php?l=zh-tw&f=EMdes010." + \
               self.date_str + "-C.csv"
        r = requests.get(link)
        lines = r.text.replace("\r", "").split("\n")
        try:
            columns_line = lines[3]
        except IndexError:
            return None
        lines = list(filter(lambda l: len(l.split('",')) > 10, lines))
        try:
            df = pd.read_csv(StringIO("\n".join(lines)), header=None)
        except pd.errors.EmptyDataError:
            return None
        df.columns = list(map(lambda l: l.replace(" ", ""), columns_line.split(",")))
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(",", ""))
        df.iloc[:, 3:] = df.iloc[:, 3:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df["date"] = pd.to_datetime(self.date).date()
        if "證券名稱" in df.columns:
            df = df[["證券代號", "date", "證券名稱", "成交量", "成交金額", "前日均價", "最後", "最高", "最低"]]
        # old format("名稱")
        else:
            df = df[["證券代號", "date", "名稱", "成交量", "成交金額", "前日均價", "最後", "最高", "最低"]]
            df = df.rename(columns={"名稱": "證券名稱"})

        df = df.rename(columns={"證券代號": "stock_id", "證券名稱": "stock_name",
                                "成交量": "turnover_vol", "成交金額": "turnover_price",
                                "前日均價": "open_price", "最後": "close_price",
                                "最高": "high_price", "最低": "low_price"})
        # solve " "
        df['stock_id'] = df['stock_id'].apply(lambda s: s[:s.index('  ')] if '  ' in s else s)
        df['stock_name'] = df['stock_name'].apply(lambda s: s[:s.index('  ')] if '  ' in s else s)
        df = df[df["stock_id"] != "合計"]
        df['market'] = 'rotc'
        return df

    def crawl_main(self):
        try:
            df = pd.concat([self.crawl_sii(), self.crawl_otc(), self.crawl_rotc()])
        except ValueError:
            return None
        return df
    
test=CrawlStockPriceTW(datetime.datetime(2020,7,1)).crawl_main()
test

,stock_id,date,stock_name,turnover_vol,turnover_price,open_price,close_price,high_price,low_price,market
0,0050,2020-07-01,元大台灣50,9.97925e+06,9.04497e+08,89.95,90.5,90.9,89.95,twse
1,0051,2020-07-01,元大中型100,16100,601402,37.02,37.48,37.53,37.02,twse
2,0052,2020-07-01,富邦科技,204514,1.43178e+07,69.4,69.9,70.3,69.4,twse
3,0053,2020-07-01,元大電子,16230,685890,41.82,42.31,42.31,41.82,twse
4,0054,2020-07-01,元大台商50,34000,855730,25,25.18,25.2,25,twse
...,...,...,...,...,...,...,...,...,...,...
243,8490,2020-07-01,利得,5000,96500,19.3,19.3,19.3,19.3,rotc
244,8491,2020-07-01,真好玩,602743,2.23455e+07,39.29,36.75,40,35.45,rotc
245,8492,2020-07-01,兔將,37007,450083,12.78,12,12.6,11.6,rotc
246,8495,2020-07-01,經緯航,47201,641983,13.65,13.9,14,13.3,rotc


In [8]:
def select_otc_id(code):
    if len(code) > 5:
        if code[-1]=="P":
            return False
        else:
            try:
                code = int(code[:5])
                if code > 10000:
                    print(2)
                    return False
                else:
                    print(1)
                    return True
            except ValueError:
                return True
    else:
        print(3)
        return True
    
select_otc_id('70123')

3


True

In [ ]:
get_pk_dict = {pk+ '__contain': pk for pk in ('a','b','c')}
get_pk_dict.keys()
# get_pk_contain_dict = {pk + '__contain': df[pk] for pk in pk_columns}

In [ ]:
z.index

In [ ]:
len()